In [ ]:
# Import libraries

from cil.framework import  AcquisitionGeometry
from cil.optimisation.algorithms import Algorithm, SPDHG
from cil.optimisation.functions import Function, L2NormSquared, BlockFunction, MixedL21Norm, IndicatorBox, LeastSquares, TotalVariation
from cil.optimisation.operators import GradientOperator, BlockOperator
from cil.optimisation.algorithms import PDHG
from cil.plugins.astra.operators import ProjectionOperator
# from cil.plugins.tigre import ProjectionOperator
from cil.plugins.astra.processors import FBP
from cil.plugins import TomoPhantom
from cil.utilities.display import show2D, show_geometry
from cil.utilities import noise
from cil.processors import Slicer

import matplotlib.pyplot as plt

import numpy as np

import importlib

In [ ]:
# Define functions
import sys
cil_path = '/u/s/cd902/GitHub/Hackathon-000-Stochastic/cil'
sys.path.append(cil_path)
import NewSubsetSumFunction
from NewSubsetSumFunction import SAGAFunction, SGDFunction, SAGFunction, SVRGFunction, LSVRGFunction
import NewFISTA
from NewFISTA import ISTA, FISTA, AdaptiveMomentumISTA

In [ ]:
# Detectors
N = 256
detectors =  N

# Angles
angles = np.linspace(0,180,180, dtype='float32')

# Setup acquisition geometry
ag = AcquisitionGeometry.create_Parallel2D()\
                        .set_angles(angles)\
                        .set_panel(detectors, pixel_size=0.1)
# Get image geometry
ig = ag.get_ImageGeometry()

# Get phantom
phantom = TomoPhantom.get_ImageData(12, ig)

In [ ]:
# Create projection operator using Astra-Toolbox. Available CPU/CPU
A = ProjectionOperator(ig, ag, device = 'gpu')

# Create an acqusition data (numerically)
sino = A.direct(phantom)

# Simulate Gaussian noise for the sinogram
gaussian_var = 0.5
gaussian_mean = 0

n1 = np.random.normal(gaussian_mean, gaussian_var, size = ag.shape)
                      
noisy_sino = ag.allocate()
noisy_sino.fill(n1 + sino.array)
noisy_sino.array[noisy_sino.array<0]=0
# noisy_sino.fill(sino.array)

# Show numerical and noisy sinograms
show2D([phantom, sino, noisy_sino], title = ['Ground Truth','Sinogram','Noisy Sinogram'], num_cols=3, cmap = 'inferno')


In [ ]:
# Setup and run the FBP algorithm
fbp_recon = FBP(ig, ag,  device = 'gpu')(noisy_sino)

# Show reconstructions
show2D([phantom, fbp_recon], 
       title = ['Ground Truth','FBP reconstruction'], 
       cmap = 'inferno', fix_range=(0,1.), size=(10,10))

# Setup subset framework for gradient-based algorithms

In [ ]:
# Define number of subsets
n_subsets = 10

# Initialize the lists containing the F_i's and A_i's
f_subsets = []
A_subsets = []

# Define F_i's and A_i's
for i in range(n_subsets):
    # Total number of angles
    n_angles = len(ag.angles)
    # Divide the data into subsets
    data_subset = Slicer(roi = {'angle' : (i,n_angles,n_subsets)})(noisy_sino)

    # Define A_i and put into list 
    ageom_subset = data_subset.geometry
    Ai = ProjectionOperator(ig, ageom_subset)
    A_subsets.append(Ai)

    # Define F_i and put into list
    fi = LeastSquares(Ai, b = data_subset)
    f_subsets.append(fi)


# Define F, G and K
F = BlockFunction(*f_subsets)
lb = 0.01
G = lb * TotalVariation(lower=0)
num_epochs = 50

# Setup subset framework for SPDHG

In [ ]:
# Initialize the lists containing the F_i's and A_i's
fpd_subsets = []
Apd_subsets = []

# Define F_i's and A_i's
for i in range(n_subsets):
    # Total number of angles
    n_angles = len(ag.angles)
    # Divide the data into subsets
    data_subset = Slicer(roi = {'angle' : (i,n_angles,n_subsets)})(noisy_sino)

    # Define A_i and put into list 
    ageom_subset = data_subset.geometry
    Ai = ProjectionOperator(ig, ageom_subset)
    Apd_subsets.append(Ai)

    # Define F_i and put into list
    fi = L2NormSquared(b = data_subset)
    fpd_subsets.append(fi)


# Define F, G and K
Fpd = BlockFunction(*fpd_subsets)
Apd = BlockOperator(*Apd_subsets)

## SAGA

In [ ]:
F_saga = SAGAFunction(F)
# admissible step-size is gamma = 1/ (3 max_i L_i)
step_size = 1 /(3* F_saga.Lmax)
initial = ig.allocate(0)
F_saga.memory_reset()
saga = ISTA(initial=initial, 
            f=F_saga,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000)
saga.run(num_epochs * n_subsets, verbose=0)

## SAGFunction, SVRGFunction, LSVRGFunction

In [ ]:
F_sag = SAGFunction(F)
initial = ig.allocate(0)
F_sag.memory_reset()
sag = ISTA(initial=initial, 
            f=F_sag,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000)
sag.run(num_epochs * n_subsets, verbose=0)

In [ ]:
F_svrg = SVRGFunction(F)
initial = ig.allocate(0)
F_svrg.memory_reset()
svrg = ISTA(initial=initial, 
            f=F_svrg,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000)
svrg.run(num_epochs * n_subsets, verbose=0)

In [ ]:
F_lsvrg = LSVRGFunction(F)
initial = ig.allocate(0)
F_lsvrg.memory_reset()
lsvrg = ISTA(initial=initial, 
            f=F_lsvrg,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000)
lsvrg.run(num_epochs * n_subsets, verbose=0)

## SPDHG

In [ ]:
initial = ig.allocate(0)
Gpd = lb * n_subsets * TotalVariation(lower=0)
spdhg =  SPDHG(f=Fpd, g=Gpd, operator=Apd, 
            update_objective_interval=n_subsets, 
            max_iteration=10000)
spdhg.run(num_epochs * n_subsets, verbose=0)

## SGDFunction

In [ ]:
F_sgd = SGDFunction(F)
initial = ig.allocate(0)
sgd = ISTA(initial=initial, 
            f=F_sgd,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000)
sgd.run(num_epochs * n_subsets, verbose=0)

## SAGA with diagonal preconditioning



In [ ]:
# With diagonal preconditioning

# D(x) = diag(x / A^T \1)
cst = A.adjoint(A.range.allocate(1.))
cst2 = ig.allocate(1.).divide(cst)
precond = lambda i, x: cst2
# precond = lambda i, x: x.divide(cst)
# csts = [Ai.adjoint(Ai.range.allocate(1.))  for Ai in A_subsets]
# precond = lambda i, x: x.divide(csts[i]) 
F_saga = SAGAFunction(F, precond=precond)
step_size = 0.1
# initial = ig.allocate(1.)
initial = ig.allocate(0.)
F_saga.memory_reset()
saga_precond = ISTA(initial=initial, 
            f=F_saga,
            g=G,
            step_size=step_size, update_objective_interval=n_subsets, 
            max_iteration=10000)
saga_precond.run(num_epochs * n_subsets, verbose=0)

## SAGA with linear acceleration

In [ ]:
F_fsaga = SAGAFunction(F)
F_saga.memory_reset()
initial = ig.allocate(0)
step_size = lambda k : (k+1)/F_fsaga.Lmax**(1/2)
momentum = lambda k : 1/((k+1)*F_fsaga.Lmax**(1/2))
fsaga = AdaptiveMomentumISTA(initial=initial, 
            f=F_fsaga,
            g=G,
            step_size=step_size, 
            momentum = momentum,
            update_objective_interval=n_subsets, 
            max_iteration=10000)
fsaga.run(num_epochs * n_subsets, verbose=0)

In [ ]:
# Look at results
show2D([saga.solution, sag.solution, svrg.solution, lsvrg.solution, spdhg.solution, saga_precond.solution,sgd.solution, fsaga.solution], 
       origin="upper", 
       title=["SAGA", "SAG", "SVRG", "L-SVRG", "SPDHG", "SAGA precond", "SGD", "F-SAGA"],
       fix_range=(0,1), num_cols=4,
       cmap='inferno')

In [ ]:
# compare results
plt.figure(figsize=(20,10))
plt.semilogy(saga.objective, label="SAGA")
plt.semilogy(sag.objective, label="SAG")
plt.semilogy(svrg.objective, label="SVRG")
plt.semilogy(lsvrg.objective, label="L-SVRG")
plt.semilogy([1/n_subsets * o for o in spdhg.objective], label="SPDHG")
plt.semilogy(saga_precond.objective, '--', label="SAGA precond",)
plt.semilogy(fsaga.objective, '--', label="F-SAGA")
plt.semilogy(sgd.objective, '--', label="SGD")
plt.legend()
plt.ylabel('Epochs')
plt.xlabel('Objective function')
plt.show()